## 구현한 YOLO 테스트

In [1]:
import numpy as np
import cv2
from functools import partial
import xmltodict
from tqdm import tqdm
import tensorflow as tf
import random
from glob import glob
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
def load_YOLO() :
    
    max_num = len(tf.keras.applications.VGG16(weights='imagenet', include_top=False,  input_shape=(224, 224, 3)).layers) # 레이어 최대 개수

    YOLO = tf.keras.models.Sequential(name = "YOLO")
    for i in range(0, max_num-1):
        YOLO.add(tf.keras.applications.VGG16(weights='imagenet', include_top=False,  input_shape=(224, 224, 3)).layers[i])

    initializer = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.01, seed=None)
    leaky_relu = tf.keras.layers.LeakyReLU(alpha=0.01)  
    regularizer = tf.keras.regularizers.l2(0.0005) # L2 규제 == weight decay.

    # 훈련된 모델은 더이상 건드리지 않기. 논문에서도 1주일 훈련시켰다고 말한 이후로 따로 언급이 없음
    for layer in YOLO.layers:
        # 훈련 X
        layer.trainable=False
    #     if hasattr(layer, 'kernel_regularizer'):
    #         setattr(layer, 'kernel_regularizer', regularizer)
        if (hasattr(layer,'activation'))==True:
            layer.activation = leaky_relu

    YOLO.add(tf.keras.layers.Conv2D(1024, (3, 3), activation=leaky_relu, kernel_initializer=initializer, kernel_regularizer = regularizer, padding = 'SAME', name = "detection_conv1", dtype='float32'))
    YOLO.add(tf.keras.layers.Conv2D(1024, (3, 3), activation=leaky_relu, kernel_initializer=initializer, kernel_regularizer = regularizer, padding = 'SAME', name = "detection_conv2", dtype='float32'))
    YOLO.add(tf.keras.layers.MaxPool2D((2, 2)))
    YOLO.add(tf.keras.layers.Conv2D(1024, (3, 3), activation=leaky_relu, kernel_initializer=initializer, kernel_regularizer = regularizer, padding = 'SAME', name = "detection_conv3", dtype='float32'))
    YOLO.add(tf.keras.layers.Conv2D(1024, (3, 3), activation=leaky_relu, kernel_initializer=initializer, kernel_regularizer = regularizer, padding = 'SAME', name = "detection_conv4", dtype='float32'))
    # Linear 부분
    YOLO.add(tf.keras.layers.Flatten())
    YOLO.add(tf.keras.layers.Dense(4096, activation=leaky_relu, kernel_initializer = initializer, kernel_regularizer = regularizer, name = "detection_linear1", dtype='float32'))
    YOLO.add(tf.keras.layers.Dropout(.5))
    # 마지막 레이어의 활성화 함수는 선형 활성화 함수인데 이건 입력값을 그대로 내보내는거라 activation을 따로 지정하지 않았다.
    YOLO.add(tf.keras.layers.Dense(1470, kernel_initializer = initializer, kernel_regularizer = regularizer, name = "detection_linear2", dtype='float32')) # 7*7*30 = 1470. 0~29 : (0, 0) 위치의 픽셀에 대한 각종 출력값, 30~59 : (1, 0) 위치의...블라블라
    YOLO.add(tf.keras.layers.Reshape((7, 7, 30), name = 'output', dtype='float32'))
    
    # 가중치 로드
    YOLO.load_weights('yolo-minkyuKim.h5')
    
    return YOLO

In [3]:
# 출력된 bbox 정보를 사진에 출력할 수 있게 처리
def process_bbox(x, y, bbox, image_size, classes_score, Classes_inDataSet) : 
    # size 처리
    bbox_x = ((32.0 * x) + (bbox[0] * 32.0)) * (image_size[0]/224.0) # 예를 들어 x = 0이면 0~32사이에 중점의 x좌표가 존재
    bbox_y = ((32.0 * y) + (bbox[1] * 32.0)) * (image_size[1]/224.0) # 예를 들어 x = 0이면 0~32사이에 중점의 x좌표가 존재
    bbox_w = bbox[2] * image_size[0] # 전체 이미지 대비 백분위
    bbox_h = bbox[3] * image_size[1] # 전체 이미지 대비 백분위
    
    min_x = int(bbox_x - bbox_w/2)
    min_y = int(bbox_y - bbox_h/2)
    max_x = int(bbox_x + bbox_w/2)
    max_y = int(bbox_y + bbox_h/2)
    
    idx_class_highest_score = np.argmax(classes_score)
    class_highest_score = classes_score[idx_class_highest_score] # 가장 높은 class score
    class_highest_score_name = Classes_inDataSet[idx_class_highest_score] # 가장 높은 score를 가진 class의 이름
    
    output_bbox = [min_x, min_y, max_x, max_y, class_highest_score, class_highest_score_name]

    return output_bbox # [x, y, w, h, class_highest_score, class_highest_score_name]로 구성된 list출력
    

In [4]:
def nms(bbox_list) : 
    nms_bbox_list = []
    for i in range(0, len(bbox_list)) :
        
        if bbox_list[i][4] > 0.5 : # class score가 0.5넘기는 것만 출력하기
            nms_bbox_list.append(bbox_list[i])
    
    return nms_bbox_list

In [5]:
def get_YOLO_output(YOLO, Image_path, Classes_inDataSet) : 
    
    image_cv = cv2.imread(Image_path)
    height, width,_ = image_cv.shape # 이미지 원래 사이즈를 얻는다. [w, h]
    image_size = [width, height]

    image_cv = cv2.resize(image_cv, (224, 224))/255
    image_cv = np.expand_dims(image_cv, axis = 0)

    image_cv = image_cv.astype('float32')

    YOLO_output = YOLO(image_cv)[0].numpy() # 계산의 간편함을 위해 numpy array로 변환. [1,7,7,30]으로 나오기 때문에 [7,7,30]으로 만들어줘야한다.
    
    bbox_list = []
    
    for y in range(0, 7) :
        for x in range(0, 7) :
            # bbox에 있는 20개의 클래스 스코어
            bbox1_class_score = YOLO_output[y][x][10:] * YOLO_output[y][x][4]
            bbox2_class_score = YOLO_output[y][x][10:] * YOLO_output[y][x][9]
        
            # bbox의 사이즈
            bbox1 = YOLO_output[y][x][0:4]
            bbox2 = YOLO_output[y][x][5:9]
            
            # 24 -> 6(box info + 가장 높게 나온 클래스 prob + 가장 높게 나온 클래스의 idx)개로 처리
            # opencv는 min_x, min_y, max_x, max_y를 원하니 x, y, w, h를 min, max 좌표로 변환
            process_bbox1 = process_bbox(x, y, bbox1, image_size, bbox1_class_score, Classes_inDataSet)
            process_bbox2 = process_bbox(x, y, bbox2, image_size, bbox2_class_score, Classes_inDataSet)
            
            bbox_list.append(process_bbox1)
            bbox_list.append(process_bbox2)
    
    nms_bbox_list = nms(bbox_list)
    # nms_bbox_list = bbox_list
    
    im_read = cv2.imread(Image_path)

    for i in range(0, len(nms_bbox_list)) :
        
        # rectangle함수를 위해 필요한 '박스의 최소 x,y 좌표'와 '박스의 최대 x,y좌표'리스트를 생성한다. 
        min_box = (nms_bbox_list[i][0], nms_bbox_list[i][1])
        max_box = (nms_bbox_list[i][2], nms_bbox_list[i][3])
        # 출력하기
        cv2.rectangle(im_read, min_box, max_box, (0, 255, 0), 1) # 박스 그리기
#         show_str = nms_bbox_list[i][5] + " : " + str(nms_bbox_list[i][4])
        show_str = nms_bbox_list[i][5] # 객체 이름만 표시
        
        # 글자 넣어주기
        text_min_box = (nms_bbox_list[i][0] + 2, nms_bbox_list[i][1] - 10)
        text_max_box = (nms_bbox_list[i][2], nms_bbox_list[i][1])
        
        cv2.rectangle(im_read, text_min_box, text_max_box, (0, 255, 0), -1) # 박스 그리기
        
        cv2.putText(im_read, show_str, (min_box[0] + 2, min_box[1] - 1), cv2.FONT_HERSHEY_PLAIN, 0.7, (0,0,0), 1)
    
    cv2.imwrite('output.jpg', im_read)

In [6]:
# 파일 경로
train_x_path = '/home/ubuntu/CUAI_2021/Advanced_Minkyu_Kim/PASCAL_VOC_2007/train/VOCdevkit/VOC2007/JPEGImages'
train_y_path = '/home/ubuntu/CUAI_2021/Advanced_Minkyu_Kim/PASCAL_VOC_2007/train/VOCdevkit/VOC2007/Annotations'

test_x_path = '/home/ubuntu/CUAI_2021/Advanced_Minkyu_Kim/PASCAL_VOC_2007/test/VOCdevkit/VOC2007/JPEGImages'
test_y_path = '/home/ubuntu/CUAI_2021/Advanced_Minkyu_Kim/PASCAL_VOC_2007/test/VOCdevkit/VOC2007/Annotations'

# 파일 경로 휙득
image_file_path_list = sorted([x for x in glob(train_x_path + '/**')])
xml_file_path_list = sorted([x for x in glob(train_y_path + '/**')])

test_image_file_path_list = sorted([x for x in glob(test_x_path + '/**')])
test_xml_file_path_list = sorted([x for x in glob(test_y_path + '/**')])

In [7]:
# 데이터셋에 존재하는 클래스가 얼마나 있는지 알아낸다
def get_Classes_inImage(xml_file_list):
    Classes_inDataSet = []

    for xml_file_path in xml_file_list: 

        f = open(xml_file_path)
        xml_file = xmltodict.parse(f.read())
        # 사진에 객체가 여러개 있을 경우
        try: 
            for obj in xml_file['annotation']['object']:
                Classes_inDataSet.append(obj['name'].lower()) # 들어있는 객체 종류를 알아낸다
        # 사진에 객체가 하나만 있을 경우
        except TypeError as e: 
            Classes_inDataSet.append(xml_file['annotation']['object']['name'].lower()) 
        f.close()

    Classes_inDataSet = list(set(Classes_inDataSet)) # set은 중복된걸 다 제거하고 유니크한? 아무튼 하나만 가져온다. 그걸 리스트로 만든다
    Classes_inDataSet.sort() # 정렬

    return Classes_inDataSet

In [8]:
YOLO = load_YOLO()
Classes_inDataSet = get_Classes_inImage(xml_file_path_list)

In [17]:
get_YOLO_output(YOLO, test_image_file_path_list[1095], Classes_inDataSet)